In [19]:
import os 
import wget
import gzip
import random
import scipy
import tensorflow as tf
from collections import defaultdict
from implicit import bpr
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

import implicit
implicit.__version__ # implicit 0.5.2 will report errors, we use 0.4.8 instead

'0.4.8'

Data is available at http://cseweb.ucsd.edu/~jmcauley/pml/data/. 
- Download and save to your own directory.
- Or, run following script to save it into `Chapter_5/data` folder automatically.

In [20]:
filenames = [
    'goodreads_fantasy.tsv', 
    'goodreads_reviews_fantasy_paranormal.json.gz', 
    'goodreads_reviews_comics_graphic.json.gz'
]

dataDir = './data'
url = 'http://jmcauley.ucsd.edu/pml_data'

if not os.path.exists(dataDir):
    os.makedirs(dataDir)
for filename in filenames:
    wget.download(os.path.join(url, filename), out=dataDir)
print("Done!")

Done!


# Latent factor model (Surprise)

Using the library's inbuilt data reader, extract tsv-formatted data

In [21]:
reader = Reader(line_format='user item rating', sep='\t')
data = Dataset.load_from_file(os.path.join(dataDir, "goodreads_fantasy.tsv"), reader=reader)

Standard latent-factor model

In [22]:
model = SVD()

Inbuilt functions to split into training and test fractions

In [23]:
trainset, testset = train_test_split(data, test_size=.25)

Fit the model and extract predictions

In [24]:
model.fit(trainset)
predictions = model.test(testset)

Estimate for a single (test) rating

In [25]:
predictions[0].est

2.6778320460327114

MSE for model predictions (test set)

In [26]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

1.1883426891039666


# Bayesian Personalized Ranking (Implicit)

In [27]:
def parseData(fname):
    for l in gzip.open(fname):
        d = eval(l)
        del d['review_text'] # Discard the reviews, to save memory when we don't use them
        yield d

Full dataset of Goodreads fantasy reviews (fairly memory-hungry, could be replaced by something smaller)

In [28]:
data = list(parseData(os.path.join(dataDir, "goodreads_reviews_fantasy_paranormal.json.gz")))

In [29]:
random.shuffle(data)

Example from the dataset

In [30]:
data[0]

{'user_id': 'd6e5c63a31f2890602e2073174a6f663',
 'book_id': '22573569',
 'review_id': 'bb0508b10fc509e06556a7fb35aae106',
 'rating': 5,
 'date_added': 'Sat Feb 28 06:32:16 -0800 2015',
 'date_updated': 'Sat Feb 28 06:32:34 -0800 2015',
 'read_at': 'Mon Feb 16 00:00:00 -0800 2015',
 'started_at': '',
 'n_votes': 1,
 'n_comments': 0}

Build a few utility data structures. Since we'll be converting the data to a sparse interaction matrix, the main structure here is to assign each user/item to an ID from 0 to nUsers/nItems.

In [31]:
userIDs,itemIDs = {},{}

for d in data:
    u,i = d['user_id'],d['book_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [32]:
nUsers,nItems

(256088, 258212)

Convert dataset to sparse matrix. Only storing positive feedback instances (i.e., rated items).

In [33]:
Xiu = scipy.sparse.lil_matrix((nItems, nUsers))
for d in data:
    Xiu[itemIDs[d['book_id']],userIDs[d['user_id']]] = 1
    
Xui = Xiu.T.tocsr()

Bayesian Personalized Ranking model with 5 latent factors

In [35]:
model = bpr.BayesianPersonalizedRanking(factors = 5, use_gpu = False)

Fit the model

In [36]:
model.fit(Xiu)

100%|██████████| 100/100 [00:14<00:00,  6.82it/s, train_auc=90.07%, skipped=1.83%]


Get recommendations for a particular user (the first one) and to get items related to (similar latent factors) to a particular item

In [37]:
recommended = model.recommend(0, Xui)
related = model.similar_items(0)

In [38]:
related

[(0, 0.9999999),
 (84589, 0.99931735),
 (188394, 0.9985939),
 (93220, 0.9978518),
 (255314, 0.9971932),
 (139253, 0.9969499),
 (77343, 0.996647),
 (40667, 0.99642795),
 (40972, 0.9963949),
 (84421, 0.99632007)]

Extract user and item factors

In [39]:
itemFactors = model.item_factors
userFactors = model.user_factors

In [40]:
itemFactors[0]

array([-0.2922052 , -0.04296844, -0.5709769 , -0.75090504,  0.07183442,
       -0.30304742], dtype=float32)

# Latent factor model (Tensorflow)

In [41]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

Goodreads comic book data

In [42]:
userIDs = {}
itemIDs = {}
interactions = []

for d in parse(os.path.join(dataDir, "goodreads_reviews_comics_graphic.json.gz")):
    u = d['user_id']
    i = d['book_id']
    r = d['rating']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

In [43]:
random.shuffle(interactions)
len(interactions)

542338

Split into train and test sets

In [44]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [45]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

Mean rating, just for initialization

In [46]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

Gradient descent optimizer, could experiment with learning rate

In [47]:
optimizer = tf.keras.optimizers.Adam(0.1)

Latent factor model tensorflow class

In [48]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

Initialize the model. Could experiment with number of factors and regularization rate.

In [49]:
modelLFM = LatentFactorModel(mu, 5, 0.00001)

2022-03-22 00:01:48.985558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 00:01:48.993280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 00:01:48.993571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 00:01:48.994273: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Training step (for the batch-based model from Chapter 5)

In [50]:
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

Run 100 iterations (really 100 batches) of gradient descent

In [51]:
for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.5396995
iteration 20, objective = 0.5216494
iteration 30, objective = 0.5249128
iteration 40, objective = 0.51959616
iteration 50, objective = 0.5176146
iteration 60, objective = 0.51533234
iteration 70, objective = 0.5095414
iteration 80, objective = 0.50680196
iteration 90, objective = 0.51118505
iteration 100, objective = 0.504537


Prediction for a particular user/item pair

In [52]:
u,i,r = interactionsTest[0]

In [53]:
modelLFM.predict(userIDs[u], itemIDs[i]).numpy()

4.9410257

# Bayesian personalized ranking (Tensorflow)

In [54]:
items = list(itemIDs.keys())

Batch-based version from Chapter 5

In [55]:
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [56]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [57]:
modelBPR = BPRbatch(5, 0.00001)

In [58]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

Run 100 batches of gradient descent

In [59]:
for i in range(100):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.5303632
iteration 20, objective = 0.47583044
iteration 30, objective = 0.4722096
iteration 40, objective = 0.47280103
iteration 50, objective = 0.47669053
iteration 60, objective = 0.47630626
iteration 70, objective = 0.47247666
iteration 80, objective = 0.4745618
iteration 90, objective = 0.47154313
iteration 100, objective = 0.46945256


Prediction for a particular user/item pair. Note that this is an unnormalized score (which can be used for ranking)

In [60]:
u,i,_ = interactionsTest[0]

In [61]:
# In this case just a score (that can be used for ranking), rather than a prediction of a rating
modelBPR.predict(userIDs[u], itemIDs[i]).numpy()

0.6619426

# Exercises

### 5.1

Adapt the latent factor model above, simply deleting any terms associated with latent factors

In [62]:
class LatentFactorModelBiasOnly(tf.keras.Model):
    def __init__(self, mu, lamb):
        super(LatentFactorModelBiasOnly, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [63]:
modelBiasOnly = LatentFactorModelBiasOnly(mu, 0.00001)

In [64]:
def trainingStepBiasOnly(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
        (grad, var) in zip(gradients, model.trainable_variables)
        if grad is not None)
    return loss.numpy()

In [65]:
for i in range(50):
    obj = trainingStepBiasOnly(modelBiasOnly, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.57053554
iteration 20, objective = 0.5389685
iteration 30, objective = 0.5308695
iteration 40, objective = 0.52404946
iteration 50, objective = 0.516751


Compute the MSEs for a model which always predicts the mean, versus one which involves bias terms

In [66]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [67]:
alwaysPredictMean = [mu for _ in interactionsTest]
labels = [r for _,_,r in interactionsTest]

In [68]:
MSE(alwaysPredictMean, labels)

1.3267613992499294

In [69]:
biasOnlyPredictions =\
    [modelBiasOnly.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in interactionsTest]

In [70]:
biasOnlyPredictions[0]

4.478474

In [71]:
MSE(biasOnlyPredictions, labels)

1.0027213071269891

### 5.2

Performance of a complete latent factor model (using the latent factor model implementation in the examples above)

In [72]:
optimizer = tf.keras.optimizers.Adam(0.1)
modelLFM = LatentFactorModel(mu, 10, 0.00001)

In [73]:
for i in range(50):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.539807
iteration 20, objective = 0.5344611
iteration 30, objective = 0.532373
iteration 40, objective = 0.53467935
iteration 50, objective = 0.5289248


In [74]:
predictions = [modelLFM.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in interactionsTest]

In [75]:
MSE(predictions, labels)

1.0097459352107794

(probably needs a little more tuning in terms of number of latent factors, learning rate, etc.)

### 5.3

Experiment with rounding the predictions

In [76]:
predictionsRounded = [int(p + 0.5) for p in predictions]

In [77]:
MSE(predictionsRounded, labels)

1.0911789652247668

Seems to result in worse performance. For a rough explanation, consider a random variable that takes a value of "1" half the time and "2" half the time; in terms of the MSE, always predicting 1.5 (and always incurring moderate errors) is preferable to always predicting either of 1 or 2 (and incurring a large error half the time).

### 5.4

Following the BPR code from examples above

In [78]:
optimizer = tf.keras.optimizers.Adam(0.1)
modelBPR = BPRbatch(10, 0.00001)

In [79]:
for i in range(50):
    obj = trainingStepBPR(modelBPR, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

In [ ]:
interactionsTestPerUser = defaultdict(set)
itemSet = set()
for u,i,_ in interactionsTest:
    interactionsTestPerUser[u].add(i)
    itemSet.add(i)

AUC implementation

In [ ]:
def AUCu(u, N): # N samples per user
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(itemSet.difference(interactionsTestPerUser[u]),N)
    for i,j in zip(positive,negative):
        si = modelBPR.predict(userIDs[u], itemIDs[i]).numpy()
        sj = modelBPR.predict(userIDs[u], itemIDs[j]).numpy()
        if si > sj:
            win += 1
    return win/N

In [ ]:
def AUC():
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(u, 10))
    return sum(av) / len(av)

In [ ]:
AUC()

0.7884310302103841